In [1]:
from bs4 import BeautifulSoup, Comment
from urllib.request import Request, urlopen
import json
import re
from pymongo import MongoClient

In [2]:
#CONNECTION_STRING = f"mongodb://localhost:27017/?readPreference=primary&directConnection=true"
#lient = MongoClient(CONNECTION_STRING)

In [3]:
#database for each portal news
def get_database():
  CONNECTION_STRING = f"mongodb://localhost:27017/?readPreference=primary&directConnection=true"
  client = MongoClient(CONNECTION_STRING)
  return client['News']

db = get_database()
kompas_db = db['Kompas']
tribun_db = db['Tribun']
detik_db = db['Detik']
cnn_db = db['Cnn']
okezone_db = db['Okezone']

In [4]:
#class crawl every portal news per 2023-06-28
def kompas_crawl():
  url = f"https://indeks.kompas.com/?site=all&date=2023-06-28&page=3"
  page = urlopen(url)
  html = page.read().decode("utf-8")
  soup = BeautifulSoup(html, "html.parser")
  paging = soup.find_all("div",{'class':'paging clearfix'})
  paging_link = paging[0].find_all('a',{'class':'paging__link'})
  #last_page = int([item.get('href').split('/')[-1] for item in paging_link][-1])
  last_page = int([item.get('href').split('/')[-1] for item in paging_link][-1].split('=')[-1])

  data = []
  for page in range(1, last_page+1):
    url = f"https://indeks.kompas.com/?site=all&date=2023-06-28&page={page}"
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    articles = soup.find_all("div", {"class":"article__list clearfix"})
    for article in articles:
      document = {
          "Title": article.find('h3', {'class':'article__title article__title--medium'}).text,
          "Url": article.find('div', {'class':'article__list__title'}).find('a')['href'],
          "Published_at": article.find('div', {'class':'article__date'}).text,
          "Tumb": article.find('div', {'class':'article__asset'}).find('img')['src']
          }
      data.append(document)
  kompas_db.insert_many(data)

  return data

def tribun_crawl():
  url = "https://www.tribunnews.com/index-news?date=2023-6-28&page=4"
  page = urlopen(url)
  html = page.read().decode("utf-8")
  soup = BeautifulSoup(html, "html.parser")
  paging = soup.find_all("div", {"class":"paging"})
  paging_link = paging[0].find_all("a")
  last_page = int([item.get('href') for item in paging_link][-1].split('=')[-1])

  data = []
  for page in range(1, last_page+1):
      url = f"https://www.tribunnews.com/index-news?date=2023-6-28&page={page}"
      page = urlopen(url)
      html = page.read().decode("utf-8")
      soup = BeautifulSoup(html, "html.parser")
      articles = soup.find_all("li", {"class":"ptb15"})

      for article in articles:
        document = {
            "Title": article.find("h3", {"class":"f16 fbo"}).find('a')['title'],
            "Url": article.find("h3", {"class":"f16 fbo"}).find('a')['href'],
            "Published_at": article.find("time", {"class":"grey"}).text
            }
        data.append(document)
  tribun_db.insert_many(data)

  return data

def detik_crawl():
  url = f"https://news.detik.com/indeks/3?date=06/28/2023"
  page = urlopen(url)
  html = page.read().decode("utf-8")
  soup = BeautifulSoup(html, "html.parser")
  paging = soup.find_all("div", {"class":"pagination text-center mgt-16 mgb-16"})
  paging_link = paging[0].find_all("a", {"class":"pagination__item"})
  last_page = [int(re.search(r"/indeks/(\d+)\?", item['href']).group(1)) for item in paging_link if re.search(r"/indeks/(\d+)\?", item['href'])]

  data = []
  for page in range(1, max(last_page)+1):
    url = f"https://news.detik.com/indeks/{page}?date=06/28/2023"
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    articles = soup.find_all("article", {"class":"list-content__item"})

    for article in articles:
      document = {
        "Title": article.find('h3', {'class':'media__title'}).text.strip().title(),
        "Url": article.find('h3', {'class':'media__title'}).find('a')['href'],
        "Published_at": article.find('div', {'class':'media__date'}).text,
        "Tumb": article.find('div', {'class':'media__image'}).find('a')['href']
      }
      data.append(document)
  detik_db.insert_many(data)

  return data

def cnn_crawl():
  url = f"https://www.cnnindonesia.com/nasional/indeks/3?date=2023/06/28"
  page = urlopen(url)
  html = page.read().decode("utf-8")
  soup = BeautifulSoup(html, "html.parser")
  hal = soup.find_all("a",{"dtr-evt":"halaman"})
  last = []
  for h in hal:
    try:
      last_ = int(h["dtr-act"].split(' ')[1])
      last.append(last_)
    except:
      pass
  data = []

  for page in range(1, max(last)+1):
    url = f"https://www.cnnindonesia.com/nasional/indeks/3/{page}?date=2023/06/28"
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    articles = soup.find_all('article', {'class':''})
    for article in articles:
      document = {
        "Title": article.find('h2', {'class':'title'}).text,
        "Url": article.find('a')['href'],
        "Published_at": article.find('span', {'class':'date'}).find(string=lambda text: isinstance(text, Comment)),
        "Tumb": article.find('span', {'class':'ratiobox ratio_16_9 box_img'}).find('img')['src']
        }
      data.append(document)
    
  cnn_db.insert_many(data)
  return data


def okezone_crawl():
  url = f"https://news.okezone.com/indeks/2023/06/28/40"
  headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36" }
  req = Request(url, headers=headers) 
  page = urlopen(req)
  html = page.read().decode("utf-8")
  soup = BeautifulSoup(html, "html.parser")
  paging = soup.find('div', {'class':'pagination-komentar'})
  links = paging.find_all('a')
  last =[int(link['data-ci-pagination-page']) for link in links]
  hal = max(last)
  data = []
  
  for page in range(1, hal+1):
    url = f"https://news.okezone.com/indeks/2023/06/28/{page}"
    headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    req = Request(url, headers=headers) 
    page = urlopen(req)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    articles = soup.find_all('li', {'class':'col-md-12 p-nol m-nol hei-index'})
    
    for article in articles:
      document = {
        "Title": article.find('h4', {'class':'f17'}).text.strip().title(),
        "Url": article.find('h4', {'class':'f17'}).find('a')['href'],
        "Published_at": article.find('time', {'class':'category-hardnews f12'}).text.strip().splitlines()[1]
        }
      data.append(document)

  okezone_db.insert_many(data)
  return data


In [5]:
#url = "https://indeks.kompas.com/?site=all&date=2023-06-28"
#urll = "https://www.tribunnews.com/index-news?date=2023-6-28&page=4"
kompas = kompas_crawl()
tribun = tribun_crawl()
detik = detik_crawl()
cnn = cnn_crawl()
okezone = okezone_crawl()